### Installation instruction:
- pip install -i https://test.pypi.org/simple/ signature-mahalanobis-knn==0.1.1
- Install sktime (modified to fix a bug, see issue 1591), esig, iisignature for signature computations: 
    - pip install git+https://github.com/sz85512678/sktime.git
    - pip install esig iisignature
- (Optional) Obtain test data from https://github.com/sz85512678/signature_mahalanobis_knn/tree/main/data

In [1]:
import os
import importlib

import pandas as pd

from sig_anomaly import SigMahaKNN

### Using anomaly detection on streams

In [2]:
DATA_DIR = "./data/"
digit = 1

train_df = pd.read_pickle(DATA_DIR + "pen_digit_train.pkl")
test_df = pd.read_pickle(DATA_DIR + "pen_digit_test.pkl")
corpus = train_df[train_df["Digit"] == digit]["Stream"].values
test_inlier = test_df[test_df["Digit"] == digit]["Stream"].values
test_outlier = test_df[test_df["Digit"] != digit]["Stream"].values

In [3]:
sig_maha_knn = SigMahaKNN.SignatureMahalanobisKNN()

In [ ]:
sig_maha_knn.fit(corpus)

In [ ]:
%%time
inlier_dists = sig_maha_knn.conformance(test_inlier)

In [ ]:
%%time
outlier_dists = sig_maha_knn.conformance(test_outlier)

In [ ]:
%%time
auc = sig_maha_knn.compute_auc_given_dists(inlier_dists, outlier_dists)
print("Auc is", auc)

### Using the variance norm as a standalong object

In [ ]:
from sktime.transformations.panel.signature_based import SignatureTransformer
from joblib import Parallel, delayed

In [41]:
mahalanobis = Distance.Mahalanobis()

In [44]:
signature_transform = SignatureTransformer(
    augmentation_list=None,
    window_name="global",
    window_depth=None,
    window_length=None,
    window_step=None,
    rescaling=None,
    sig_tfm="signature",
    depth=2,
)

In [48]:
sigs = Parallel(n_jobs=-2)(
    delayed(signature_transform.fit_transform)(corpus[i]) for i in range(len(corpus))
)
sigs = pd.concat(sigs)

In [49]:
mahalanobis.fit(sigs)

In [54]:
y1 = test_inlier[0]
sig_y1 = signature_transform.fit_transform(y1)

In [55]:
y2 = corpus[0]
sig_y2 = signature_transform.fit_transform(y2)

In [57]:
mahalanobis.distance(sig_y1, sig_y2)

0
0  0.012913